# Node.js 교과서 7장
# < MySQL >

## 7.4 ~ 7.5 SQL문 작성은 아래를 참고
![](img/MySQL1.png)
![](img/MySQL2.png)

## 7.6 시퀄라이즈 사용하기
시퀄라이즈는 자바스크립트 객체와 데이터베이스 릴레이션을 매핑해주는 ORM(Object-Relational Mapping)도구이다.

In [1]:
%%writefile code/7/package.json
{
  "name": "learn-sequelize",
  "version": "0.0.1",
  "description": "시퀄라이즈를 배우자",
  "main": "app.js",
  "scripts": {
    "start": "nodemon app"
  },
  "author": "hyejin",
  "license": "MIT"
}

Writing code/7/package.json


**모듈 설치**
> npm i express morgan nunjucks sequelize sequelize-cli mysql2

 - sequelize-cli: 시퀄라이즈 명령어를 실행하기 위한 패키지 
 - mysql2: MySQL과 시퀄라이즈를 이어주는 드라이버
 
> npm i -D nodemon

> npx sequelize init

config, models, migations, seeders 폴더가 생성된다.  
models 폴더 안에 index.js가 있는데, sequelize-cli 설치 후 자동생성되는 코드가 들어가있다.  
그대로 사용하면 에러가 발생하고 필요없는 부분도 많으므로 아래와 같이 수정한다.

In [7]:
%%writefile code/7/models/index.js

const Sequelize = require('sequelize');

const env = process.env.NODE_ENV || 'development';
const config = require('../config/config')[env];
const db = {};

const sequelize = new Sequelize(config.database, config.username, config.password, config);

db.sequelize = sequelize;

module.exports = db;

Overwriting code/7/models/index.js


### 7.6.1 MySQL 연결하기

MySQL연동을 위해서는 config 폴더에 config.json 정보를 넣어주어야한다.  
MySQL 커넥션과 일치하도록 development.password와 development.database를 수정한다.

```json
{
  "development": {
    "username": "root",
    "password": "비밀번호",   < MySQL 접속 비밀번호를 넣는다.
    "database": "nodejs",     < 생성해둔 스키마 이름을 넣는다.
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "test": {
    "username": "root",
    "password": null,
    "database": "database_test",
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "production": {
    "username": "root",
    "password": null,
    "database": "database_production",
    "host": "127.0.0.1",
    "dialect": "mysql"
  }
}
```

In [8]:
%%writefile code/7/app.js

const express = require('express');
const path = require('path');
const morgan = require('morgan');
const nunjucks = require('nunjucks');

const { sequelize } = require('./models');

const app = express();
app.set('port', process.env.PORT || 3001);
app.set('view engine', 'html');
nunjucks.configure('views', {
  express: app,
  watch: true,
});
sequelize.sync({force:false})
  .then(() => {
    console.log('데이터베이스 연결 성공~');
  })
  .catch((err) => {
    console.log(err);
  });

app.use(morgan('dev'));
app.use(express.static(path.join(__dirname, 'public')));
app.use(express.json());
app.use(express.urlencoded({ extended: false }));

app.use((req, res, next) => {
  const error = new Error(`${req.method} ${req.url} 라우터가 없습니다.`);
  error.status = 404;
  next(error);
});

app.use((err, req, res, next) => {
  res.locals.message = err.message;
  res.locals.error = process.env.NODE_ENV !== 'production' ? err:{};
  res.status(err.status || 500);
  res.render('error');
});

app.listen(app.get('port'), () => {
  console.log(app.get('port'), ' 포트에서 대기중');
});


Overwriting code/7/app.js


> npm start  

```
3001  포트에서 대기중  
Executing (default): SELECT 1+1 AS result  
데이터베이스 연결 성공~
```
잘 연결된다

### 7.6.2 모델 정의하기
models에 user과 comment를 정의하자

In [16]:
%%writefile code/7/models/user.js
const Sequelize = require('sequelize');

module.exports = class User extends Sequelize.Model {
  static init(sequelize) {
    return super.init({
      // 시퀄라이즈에서는 primary key를 자동으로 생성해준다.
      name: {
        type: Sequelize.STRING(20),
        allowNull: false,
        unique: true,
      },
      age: {
        type: Sequelize.INTEGER.UNSIGNED,
        allowNull: false,
      },
      married: {
        type: Sequelize.BOOLEAN,
        allowNull: false,
      },
      comment: {
        type: Sequelize.TEXT,
        allowNull: true,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: false,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false, //timestamps가 true이면 createdAt(데이터생성시간), updatedAt(데이터수정시간)을 생성한다.
      underscored: false, //true이면, 테이블명과 컬럼명을 snake_case로 바꿈 (false일때는 camelCase)
      modelName: 'User', //노드에서 사용할 모델 이름
      tableName: 'users', //데이터베이스에서의 테이블 이름
      paranoid: false, //true이면 deletedAt에 삭제 시간을 기록한다. 삭제하지 않은 데이터의 deletedAt은 null이된다. (soft delete 처리)
      charset: 'utf8',
      collate: 'utf8_general_ci',
    });
  }
  static associate(db){}
};

Writing code/7/models/user.js


In [17]:
%%writefile code/7/models/comment.js

const Sequelize = require('sequelize');

module.exports = class Comment extends Sequelize.Model {
  static init(sequelize) {
    return super.init({
      comment: {
        type: Sequelize.STRING(100),
        allowNull: false,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: true,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false,
      modelName: 'Comment',
      tableName: 'comments',
      paranoid: false,
      charset: 'utf8mb4',
      collate: 'utf8mb4_general_ci',
    });
  }
  static associate(db){}
};

Writing code/7/models/comment.js


모델을 생성했다면 models/index.js와 연결한다.

In [20]:
%%writefile code/7/models/index.js

const Sequelize = require('sequelize');
const User = require('./user');
const Comment = require('./comment');

const env = process.env.NODE_ENV || 'development';
const config = require('../config/config')[env];
const db = {};

const sequelize = new Sequelize(config.database, config.username, config.password, config);

db.sequelize = sequelize;

db.User = User;
db.Comment = Comment;

User.init(sequelize);
Comment.init(sequelize);

User.associate(db);
Comment.associate(db);

module.exports = db;

Overwriting code/7/models/index.js


### 7.6.3 관계 정의하기
**관계**
1. 1:N (일대다)
 - hasMany와 belongTo 메서드로 표현한다.
 - users(사용자)는 여러개의 comment(댓글)을 남길 수 있다.
 - 댓글을 남긴 사용자는 commenter라고 하자
 ```js
// models/user.js
    static associate(db){
        db.User.hasMany(db.Comment, {foreignKey: 'commenter', sourceKey: 'id'});
    } //user의 id를 comment가 commenter로 참조한다.
 ```
 - belongTo 메서드: 다른 모델의 정보가 들어가는 테이블에 사용함
 ```js
// models/comment.js
   static associate(db){
       db.User.belongsTo(db.User, {foreignKey: 'commenter', targetKey: 'id'});
   } //comment가 user의 id를 commenter로 참조한다.
 ```
2. 1:1 (일대일)
3. N:M (다대다)